In [1]:
import plotly.offline as pyo
import plotly.graph_objs as go
import plotly.plotly as py
import seaborn as sns
import sqlite3 as sq
import pandas as pd
import numpy as np
import dash
import dash_core_components as dcc
import dash_html_components as html
import pandas_profiling as pp
import datetime as dt
import plotly
%matplotlib inline
pyo.init_notebook_mode(connected=True)

/Users/yuriysimonov/anaconda3/lib/python3.6/site-packages/pandas_profiling/base.py:20: UserWarning:


This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/Users/yuriysimonov/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/yuriysimonov/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/yuriysimonov/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/yuriysimonov/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/yuriysimonov/anaconda3/lib/python3.6/site-package

In [2]:
import sqlite3 as sq
import pandas as pd

In [7]:
day_w_dict = {0:'пн',1:'вт',2:'ср',3:'чт',4:'пт',5:'сб',6:'вс'}
day_m_dict = {'January':0, 'February':1, 'March':2, 'April':3, 'May':4, 'June':5, 'July':6,
              'August':7, 'September':8, 'October':9, 'November':10, 'December':11}

In [19]:
def age_taker(x):
    if x is None:
        return np.nan
    elif np.round((dt.datetime.now() - dt.datetime.strptime(x[:10],'%d.%m.%Y')).days/365,2)<0 or \
    np.round((dt.datetime.now() - dt.datetime.strptime(x[:10],'%d.%m.%Y')).days/365,2)>100:
        return np.nan
    else:
        return np.round((dt.datetime.now() - dt.datetime.strptime(x[:10],'%d.%m.%Y')).days/365,2)
def age_grouper(x):
    if x>=0:
        dict_age = {0: '0-4', 1: '4-14', 2: '14-21',
                    3: '21-35', 4: '35-50', 5: '50-'}
        return dict_age[int((x > 4)+(x > 14)+(x > 21)+(x > 35)+(x > 50))]
    else:
        return 'Не указан'

# Finance

## Aqv

In [3]:
rerquest_aqv = '''select
   ca.CHECK_DATE
  ,bra.SUM
  ,oa.ORDER_ID,
  ca.CHECK_ID
from check_aqv ca
inner join bill_row_aqv bra on ca.CHECK_ID = bra.CHECK_ID
inner join order_aqv oa on oa.BILL_ROW_ID = bra.BILL_ROW_ID'''

In [4]:
with sq.connect('isd_base.db') as conn:
    finance_aqv = pd.read_sql(rerquest_aqv, conn)

In [5]:
# correct types
finance_aqv['CHECK_DATE'] = pd.to_datetime(finance_aqv['CHECK_DATE'])
finance_aqv['SUM'] = pd.to_numeric(finance_aqv['SUM'])

In [6]:
finance_aqv['D_N'] = finance_aqv.CHECK_DATE.dt.dayofweek
finance_aqv['M'] = finance_aqv.CHECK_DATE.dt.month_name()
finance_aqv['H'] = finance_aqv.CHECK_DATE.dt.hour

In [8]:
finance_aqv['D'] = finance_aqv.D_N.map(day_w_dict)
finance_aqv['M_N'] = finance_aqv.M.map(day_m_dict)

In [9]:
finance_aqv.to_pickle('finance_aqv.pcl')

## glk

In [10]:
rerquest_glk = '''select
   ca.CHECK_DATE
  ,bra.SUM
  ,oa.ORDER_ID,
  ca.CHECK_ID
from check_glk ca
inner join bill_row_glk bra on ca.CHECK_ID = bra.CHECK_ID
inner join order_glk oa on oa.BILL_ROW_ID = bra.BILL_ROW_ID'''

In [11]:
with sq.connect('isd_base.db') as conn:
    finance_glk = pd.read_sql(rerquest_glk, conn)

CPU times: user 11.2 s, sys: 3.25 s, total: 14.5 s
Wall time: 15.2 s


In [12]:
# correct types
finance_glk['CHECK_DATE'] = pd.to_datetime(finance_glk['CHECK_DATE'])
finance_glk['SUM'] = pd.to_numeric(finance_glk['SUM'])

finance_glk['D_N'] = finance_glk.CHECK_DATE.dt.dayofweek
finance_glk['M'] = finance_glk.CHECK_DATE.dt.month_name()
finance_glk['H'] = finance_glk.CHECK_DATE.dt.hour

finance_glk['D'] = finance_glk.D_N.map(day_w_dict)
finance_glk['M_N'] = finance_glk.M.map(day_m_dict)

In [13]:
finance_glk.to_pickle('finance_glk.pcl')

# Social

## aqv

In [14]:
rerquest_soc_aqv = '''select 
   oa.ORDER_ID
  ,oa.START_DATE
  ,oa.END_DATE
  ,cla.SEX
  ,bra.SUM
  ,cla.BIRTH_DATE
  ,ca.CHECK_DATE
from order_aqv oa 
left join clients_aqv cla on oa.CLIENT_ID = cla.CLIENT_ID
left join bill_row_aqv bra on oa.BILL_ROW_ID = bra.BILL_ROW_ID
left join check_aqv ca on ca.CHECK_ID = bra.CHECK_ID'''

In [15]:
with sq.connect('isd_base.db') as conn:
    social_aqv = pd.read_sql(rerquest_soc_aqv, conn)

In [16]:
# correct types
social_aqv['CHECK_DATE'] = pd.to_datetime(social_aqv['CHECK_DATE'])
social_aqv['START_DATE'] = pd.to_datetime(social_aqv['START_DATE'])
social_aqv['END_DATE'] = pd.to_datetime(social_aqv['END_DATE'])
social_aqv['SUM'] = pd.to_numeric(social_aqv['SUM'])

In [17]:
social_aqv['D_N'] = social_aqv.CHECK_DATE.dt.dayofweek
social_aqv['M'] = social_aqv.CHECK_DATE.dt.month_name()
social_aqv['H'] = social_aqv.CHECK_DATE.dt.hour

In [18]:
social_aqv['D'] = social_aqv.D_N.map(day_w_dict)
social_aqv['M_N'] = social_aqv.M.map(day_m_dict)

In [20]:
social_aqv['AGE'] = social_aqv['BIRTH_DATE'].replace('',None).map(age_taker)
social_aqv['SESSION_TERM'] = (social_aqv.END_DATE - social_aqv.START_DATE).map(lambda x: x.seconds/3600)
social_aqv['SEX'] = social_aqv['SEX'].map({'1':'М','2':'Ж',None:'Не указан'})
social_aqv['AGE_GROUP'] = social_aqv['AGE'].map(age_grouper)

In [21]:
social_aqv.to_pickle('social_aqv.pcl')

## glk

In [22]:
rerquest_soc_glk = '''select 
   oa.ORDER_ID
  ,oa.START_DATE
  ,oa.END_DATE
  ,cla.SEX
  ,bra.SUM
  ,cla.BIRTH_DATE
  ,ca.CHECK_DATE
from order_glk oa 
left join clients_glk cla on oa.CLIENT_ID = cla.CLIENT_ID
left join bill_row_glk bra on oa.BILL_ROW_ID = bra.BILL_ROW_ID
left join check_glk ca on ca.CHECK_ID = bra.CHECK_ID'''

In [23]:
with sq.connect('isd_base.db') as conn:
    social_glk = pd.read_sql(rerquest_soc_glk, conn)

In [24]:
# correct types
social_glk['CHECK_DATE'] = pd.to_datetime(social_glk['CHECK_DATE'])
social_glk['START_DATE'] = pd.to_datetime(social_glk['START_DATE'])
social_glk['END_DATE'] = pd.to_datetime(social_glk['END_DATE'])
social_glk['SUM'] = pd.to_numeric(social_glk['SUM'])

In [25]:
social_glk['D_N'] = social_glk.CHECK_DATE.dt.dayofweek
social_glk['M'] = social_glk.CHECK_DATE.dt.month_name()
social_glk['H'] = social_glk.CHECK_DATE.dt.hour
social_glk['D'] = social_glk.D_N.map(day_w_dict)
social_glk['M_N'] = social_glk.M.map(day_m_dict)

In [26]:
social_glk['AGE'] = social_glk['BIRTH_DATE'].replace('',None).map(age_taker)
social_glk['SESSION_TERM'] = (social_glk.END_DATE - social_glk.START_DATE).map(lambda x: x.seconds/3600)
social_glk['SEX'] = social_glk['SEX'].map({'1':'М','2':'Ж',None:'Не указан'})

In [27]:
social_glk['AGE_GROUP'] = social_glk['AGE'].map(age_grouper)

In [28]:
social_glk.to_pickle('social_glk.pcl')

# Tech

## aqv

In [29]:
request_tech_aqv = """select 
  DEVICE,
  DATE,
  EVENT
from transaction_aqv
 where DEVICE like '%Турникет%'"""

In [32]:
with sq.connect('isd_base.db') as conn:
    tech_aqv = pd.read_sql(request_tech_aqv, conn)

In [33]:
# correct types
tech_aqv['DATE'] = pd.to_datetime(tech_aqv['DATE'])

In [34]:
tech_aqv['D_N'] = tech_aqv.DATE.dt.dayofweek
tech_aqv['M'] = tech_aqv.DATE.dt.month_name()
tech_aqv['H'] = tech_aqv.DATE.dt.hour

In [35]:
tech_aqv['D'] = tech_aqv.D_N.map(day_w_dict)
tech_aqv['M_N'] = tech_aqv.M.map(day_m_dict)

In [36]:
tech_aqv.to_pickle('tech_aqv.pcl')

## glk

In [37]:
request_tech_glk = """select 
  DEVICE,
  DATE,
  EVENT
from transaction_glk
 where DEVICE like '%Турникет%'"""

In [38]:
with sq.connect('isd_base.db') as conn:
    tech_glk = pd.read_sql(request_tech_glk, conn)

In [39]:
# correct types
tech_glk['DATE'] = pd.to_datetime(tech_glk['DATE'])

In [40]:
%%time
tech_glk['D_N'] = tech_glk.DATE.dt.dayofweek
tech_glk['M'] = tech_glk.DATE.dt.month_name()
tech_glk['H'] = tech_glk.DATE.dt.hour

CPU times: user 2.68 s, sys: 365 ms, total: 3.04 s
Wall time: 3.2 s


In [41]:
tech_glk['D'] = tech_glk.D_N.map(day_w_dict)
tech_glk['M_N'] = tech_glk.M.map(day_m_dict)

In [42]:
tech_glk.to_pickle('tech_glk.pcl')